In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem

dir_data='/Users/ahenny/'
dir1='/Volumes/LaCie/'
dir2='/Volumes/Extreme Pro/'

In [ ]:
ds=xr.open_dataset('/Users/ahenny/Downloads/REGEN_AllStns_V1-2019_'+str(1981)+'.nc')
precip=ds['p']#.where(quality==1)

regen_threshold_final=xr.zeros_like(precip)
precip.close()
ds.close()

dk=xr.Dataset()
dk['threshold']=(('time','lat','lon'),regen_threshold_final.values)
dk.coords['time']=regen_threshold_final.time
dk.coords['lat']=regen_threshold_final.lat
dk.coords['lon']=regen_threshold_final.lon

try:
    os.remove(dir_data+'regenfull_threshold_initial_98_all.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'regenfull_threshold_initial_98_all.nc',mode='w',format='NETCDF4')

dk.close()
regen_threshold_final.close()

In [ ]:
####ALL DAYS (INCLUDING DRY)
yrs=np.arange(1991,2017,1)

lon_bins=np.arange(-180,210,30)
lat_bins=np.arange(-90,120,30)
        
ds=xr.open_dataset('/Users/ahenny/Downloads/REGEN_AllStns_V1-2019_1950-2016_QualityMask.nc')
quality=ds['p']

dates_all=[]
for i in range(len(yrs)):
    year=yrs[i]
    year_length=365
    if year%4==0:
        year_length=366
            
    date_start=dt.datetime(year,1,1,0)
    date_range=[date_start+dt.timedelta(days=x) for x in range(year_length)]
    dates_all=dates_all+date_range

for j in range(len(lat_bins)-1):
    for k in range(len(lon_bins)-1):
        print((j,k))
        #ds1.close()
        ds1=xr.open_dataset(dir_data+'regenfull_threshold_initial_98_all.nc')
        threshold=ds1['threshold'].values
        
        lat_range=[x for x in ds1.lat.values if lat_bins[j]<=x<lat_bins[j+1]]
        lon_range=[x for x in ds1.lon.values if lon_bins[k]<=x<lon_bins[k+1]]
        
        zipped_lat=list(zip(np.arange(ds1.lat.size),ds1.lat.values.tolist()))
        zipped_lon=list(zip(np.arange(ds1.lon.size),ds1.lon.values.tolist()))

        select_lat_indices=[x[0] for x in zipped_lat if x[1] in lat_range]
        select_lon_indices=[x[0] for x in zipped_lon if x[1] in lon_range]
            
        min_lat_index=min(select_lat_indices)
        min_lon_index=min(select_lon_indices)
        max_lat_index=max(select_lat_indices)+1
        max_lon_index=max(select_lon_indices)+1
        
        threshold_test=np.sum(threshold[:,min_lat_index:max_lat_index,min_lon_index:max_lon_index])
        
        if threshold_test==0:
        
            for i in range(len(yrs)):
                year=yrs[i]
                #print(year)

                ds=xr.open_dataset('/Users/ahenny/Downloads/REGEN_AllStns_V1-2019_'+str(year)+'.nc')
                precip=ds['p']#.where(quality==1)
                precip=precip.sel(lat=lat_range,lon=lon_range).values

                if i==0:
                    precip_concat=precip
                else:
                    precip_concat=np.concatenate((precip_concat,precip),axis=0)    
            
            dk=xr.Dataset()
            dk['precip']=(('time','lat','lon'),precip_concat)
            dk.coords['time']=dates_all
            dk.coords['lat']=lat_range
            dk.coords['lon']=lon_range

            precip=dk['precip']#.fillna(0)
            
            for i in range(365):
                #print(i)
                ds=xr.open_dataset(dir_data+'holding/threshold_window_'+str(i+1)+'.nc')
                years=ds['years'].values.tolist()
                months=ds['months'].values.tolist()
                days=ds['days'].values.tolist()
                overlap_list=[dt.datetime(x,y,z) for x,y,z in zip(years,months,days)]
                
                new_dates=[]#add earlier years
                for i1 in range(365):
                    date=dt.datetime(2003,1,1)+dt.timedelta(days=i1)
                    if date in overlap_list:
                        new_dates=new_dates+[dt.datetime(1950+x,date.month,date.day) for x in range(29)]

                date=dt.datetime(2004,2,29)
                if date in overlap_list:
                    new_dates=new_dates+[dt.datetime(1952+4*x,2,29) for x in range(7)]

                overlap_list=overlap_list+new_dates
                
                overlap_adjust=[x for x in overlap_list if pd.to_datetime(x) in precip.time.values]

                precip_select=precip.sel(time=overlap_adjust).values#precip within this 61-day window
                threshold_array=np.nanquantile(precip_select,0.98,axis=0)
                threshold[i,min_lat_index:max_lat_index,min_lon_index:max_lon_index]=threshold_array#.values

            dk=xr.Dataset()
            dk['threshold']=(('time','lat','lon'),threshold)
            dk.coords['time']=ds1.time
            dk.coords['lat']=ds1.lat
            dk.coords['lon']=ds1.lon

            try:
                os.remove(dir_data+'regenfull_threshold_initial_98_all.nc')
            except OSError:
                pass
            dk.to_netcdf(dir_data+'regenfull_threshold_initial_98_all.nc',mode='w',format='NETCDF4')
            dk.close()
            ds.close()
            ds1.close()

In [ ]:
#ds.close()
ds=xr.open_dataset(dir_data+'regenfull_threshold_initial_98_all.nc')
threshold=ds['threshold']
print(threshold)

threshold_new=xr.zeros_like(threshold)
for i1 in range(threshold.lat.size):
    print(i1)
    for i2 in range(threshold.lon.size):
        time_series=threshold[:,i1,i2].values.tolist()
        smoothed=[]
        smoothed1=[]
        
        X=20#41-day running mean
        for j in range(len(time_series)):
            if j<X:
                #print(np.arange(len(time_series)-1*X+j,len(time_series),1))
                #print(np.arange(0,j+X+1,1))
                window=time_series[0:j+X+1]+time_series[-1*X+j:]
            elif j>len(time_series)-X-1:
                #print(j)
                #print(np.arange(j-X,len(time_series),1))
                #print(np.arange(0,j+X-len(time_series)+1,1))
                window=time_series[j-X:]+time_series[0:j+X-len(time_series)+1]
                #sys.exit()
            else:
                #print(np.arange(j-X,j+X+1,1))
                window=time_series[j-X:j+X+1]
                #sys.exit()

            window_mean=float(sum(window))/float(len(window))
            smoothed1.append(window_mean)
            
            
        for j in range(len(smoothed1)):
            if j<X:
                #print(np.arange(len(time_series)-1*X+j,len(time_series),1))
                #print(np.arange(0,j+X+1,1))
                window=smoothed1[0:j+X+1]+smoothed1[-1*X+j:]
            elif j>len(smoothed1)-X-1:
                #print(j)
                #print(np.arange(j-X,len(time_series),1))
                #print(np.arange(0,j+X-len(time_series)+1,1))
                window=smoothed1[j-X:]+smoothed1[0:j+X-len(smoothed1)+1]
                #sys.exit()
            else:
                #print(np.arange(j-X,j+X+1,1))
                window=smoothed1[j-X:j+X+1]
                #sys.exit()

            window_mean=float(sum(window))/float(len(window))
            smoothed.append(window_mean)
            
        #sys.exit()
            
        threshold_new[:,i1,i2]=smoothed

In [ ]:
dk=xr.Dataset()
dk['threshold']=(('time','lat','lon'),threshold_new.values)
dk.coords['time']=threshold_new.time
dk.coords['lat']=threshold_new.lat
dk.coords['lon']=threshold_new.lon
try:
    os.remove(dir_data+'regenfull_threshold_final_98_all.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'regenfull_threshold_final_98_all.nc',mode='w',format='NETCDF4')